In [1]:
import requests 

from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from time import sleep
import pandas as pd
import numpy as np

# Para visualizar el progreso:
import tqdm

# FUNCIONES SCRAPING:

- FUNCIÓN INFO:

In [2]:
def scrape_info(user_agent, urls, scroll):
    
    # Options:
    opts = Options()
    opts.add_argument("--disable-extensions")
    opts.add_argument(f"User-Agent={user_agent}")
    
    # Initialize:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    sleep(5)
    
    info_productos = []
    
    inicio = 1
    for url in urls:
        
        browser.get(url)
        sleep(5)###########################################################################
        
        #Acepto cookies:
        if inicio == 1:            
            browser.find_element(By.CLASS_NAME, "buttons_aviso_cookies").click()
            sleep(5)########################################################################
            
        # Scroll de la página hasta el final para tener todos los productos visibles:
        if scroll:        
            last_height = browser.execute_script("return document.documentElement.scrollHeight")
            while True:
                # Scroll:
                browser.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
                sleep(3)
                # Calcula la altura del nuevo scroll y la compara con la del último:
                new_height = browser.execute_script("return document.documentElement.scrollHeight")

                if new_height == last_height:
                    print("Ya no hay más página.")
                    break
                last_height = new_height    
                
        # Saco la info de cada producto:
        soup = BeautifulSoup(browser.page_source, "html.parser")
        sleep(5)
        
        productos = productos = soup.find("div", class_= "contenedor_productos").find_all("div", class_= "mini_contenedor_grupo")
        cat = soup.find("div", class_= "div_titulo").find("h1").text
        
        for producto in tqdm.tqdm(productos):
        
            categoria = cat
            url_producto = producto.find_all("a")[1]["href"]

            try:
                marca = producto.find("div", class_= "container_info_producto").find("div", class_= "nombre_marca").text
            except:
                marca = "Sin marca"

            try:
                nombre = producto.find("div", class_= "container_info_producto").find("div", class_= "nombre_grupo").text
            except:
                nombre = "Sin nombre"

            try:
                tipo = producto.find("div", class_= "container_info_producto").find("div", class_= "tipo_producto").text
            except:
                tipo = "Sin categoria"           

            try:    
                precio_sin_desc = producto.find("div", class_= "contenedor_precios").find_all("div", class_= "precio")[0].find("div", class_= "txt_precio_real").text
            except:
                precio_sin_desc = "Sin precio"
                
            try:
                precio_con_desc = producto.find("div", class_= "contenedor_precios").find_all("div", class_= "precio")[0].find("div", class_= "txt_precio_venta").text
            except:
                precio_con_desc = "Sin precio"
                    
            try:
                descuento = 100 - 100*(float(precio_con_desc.replace("€", "").replace(",", "."))/float(precio_sin_desc.replace("€", "").replace(",", ".")))
            except:
                descuento = "No tiene"
            
            response_url = requests.get(url_producto)
            soup_url = BeautifulSoup(response_url.text, "html.parser")
            try:
                info_rating = soup_url.find("div", class_= "contenedor-info-rating-total").find("div", class_= "contenedor-cantidad-opiniones").find("span").text.replace("Opinión", "").replace("Opiniones", "").split("/5")
            except:
                valoracion = "Sin rating"
                num_valoraciones = "Sin valoraciones"

            valoracion = info_rating[0]
            num_valoraciones = info_rating[1].strip().replace("(", "").replace(")", "")

                

            info_productos.append([categoria, marca, nombre, tipo, precio_sin_desc, descuento, precio_con_desc, valoracion, num_valoraciones])
        
        
        
        
   
        inicio += 1
    ############################################################################################################    
    browser.close()
    df_info_productos = pd.DataFrame(info_productos, columns= ["categoria", "marca", "nombre", "tipo", "precio_sin_desc", "descuento", "precio_con_desc", "valoracion", "num_valoraciones"])
    return df_info_productos

# Scrapeo la info de todos los productos de cada página:

In [3]:
# PONER EL USER-AGENT Y LOS ELEMENTOS DE BUSQUEDA:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

urls = ["https://perfumes24h.com/perfumes-hombre/", "https://perfumes24h.com/perfumes-mujer/"]

In [4]:
df_info_productos = scrape_info(user_agent, urls, True)

C:\Users\regue\AppData\Local\Temp\ipykernel_4616\1832564813.py:9: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path = path_driver, chrome_options=opts)


Ya no hay más página.


100%|██████████| 1135/1135 [13:22<00:00,  1.41it/s]


Ya no hay más página.


100%|██████████| 1977/1977 [23:14<00:00,  1.42it/s]


## CSV:

In [5]:
df_info_productos.to_csv("datos_perfumes_perfumes24h.csv", index="False", sep= ",")

In [103]:
######################################################################################################################################################

In [104]:
######################################################################################################################################################